In [ ]:
!pip install pgmpy
!pip install numpy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

Preprocessing of our data to feed to the model


In [ ]:
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv('customer_churn.csv')

# =================================================================
# 1. Remove Unique Identifier
# =================================================================
df.drop('customerID', axis=1, inplace=True)

# =================================================================
# 2. Convert Binary Numeric to Categorical
# =================================================================
df['SeniorCitizen'] = df['SeniorCitizen'].map({0: 'No', 1: 'Yes'})

# =================================================================
# 3. Handle Missing Values in TotalCharges
# =================================================================
# Convert to numeric, coercing errors to NaN
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Fill missing values (new customers with tenure=0)
df['TotalCharges'] = df['TotalCharges'].fillna(0)

# =================================================================
# 4. Discretize Numerical Variables
# =================================================================
# Tenure (months)
df['tenure'] = pd.cut(df['tenure'],
                     bins=[0, 12, 24, float('inf')],
                     labels=['Short', 'Medium', 'Long'],
                     right=False)

# Monthly Charges
df['MonthlyCharges'] = pd.qcut(df['MonthlyCharges'],
                              q=3,
                              labels=['Low', 'Medium', 'High'])

# Total Charges
df['TotalCharges'] = pd.qcut(df['TotalCharges'],
                            q=3,
                            labels=['Low', 'Medium', 'High'])

# =================================================================
# 5. Process Categorical Variables
# =================================================================
categorical_cols = ['gender', 'Partner', 'Dependents', 'PhoneService',
                   'MultipleLines', 'InternetService', 'OnlineSecurity',
                   'OnlineBackup', 'DeviceProtection', 'TechSupport',
                   'StreamingTV', 'StreamingMovies', 'Contract',
                   'PaperlessBilling', 'PaymentMethod', 'Churn']

# Convert to categorical data type
for col in categorical_cols:
    df[col] = df[col].astype('category')

# =================================================================
# 6. Final Checks
# =================================================================
print(df.head())
print("\nData Types:")
print(df.dtypes)
print("\nMissing Values:")
print(df.isnull().sum())

# Optional: Save preprocessed data
df.to_csv('preprocessed_churn.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'customer_churn.csv'

Learn the inherent BN DAG structure of our data and displaying it



In [ ]:
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import HillClimbSearch, BicScore
from pgmpy.estimators import MaximumLikelihoodEstimator
import networkx as nx
import matplotlib.pyplot as plt

# =================================================================
# 1. Import Preprocessed Data
# =================================================================
# Load the preprocessed data
df = pd.read_csv('preprocessed_churn.csv')

# Display the first few rows
print("Preprocessed Data:")
print(df.head())

# =================================================================
# 2. Learn the Bayesian Network Structure
# =================================================================
# Use HillClimbSearch with BIC score to learn the structure
hc = HillClimbSearch(df)
best_model = hc.estimate(scoring_method=BicScore(df))

# Display the learned structure
print("\nLearned Structure (Edges):")
print(best_model.edges())




In [ ]:

# =================================================================
# 4. Visualize the Bayesian Network Structure
# =================================================================
# Create a directed graph using networkx
G = nx.DiGraph()
G.add_edges_from(best_model.edges())

# Draw the graph
plt.figure(figsize=(10, 8))
pos = nx.kamada_kawai_layout(G)  # Kamada-Kawai layout
options = {
    'node_color': 'skyblue',
    'node_size':5000,
    'width': 1,
    'with_labels':True,
    'arrowsize':10,
    'font_size':8,
    'font_color': 'black'
}
nx.draw(G, pos, **options)
plt.title("Learned Bayesian Network Structure", fontsize=14)
plt.show()


In [ ]:
# =================================================================
# 4. Create and Fit the Bayesian Model
# =================================================================
# Initialize the BayesianModel with the learned structure
model = BayesianNetwork(best_model.edges())

# Fit the model to the data using Maximum Likelihood Estimation
model.fit(df, estimator=MaximumLikelihoodEstimator)

In [ ]:
# =================================================================
# 5. Run a Sample Query
# =================================================================
# Create a DataFrame with the evidence
evidence_df = pd.DataFrame([{
    'tenure': 'Short',
    'Contract': 'Month-to-month',
    'MonthlyCharges': 'High'
}])

# Predict the probability of churn given the evidence
query = model.predict_probability(evidence_df)

# Extract only the Churn probabilities
churn_probabilities = query[['Churn_No', 'Churn_Yes']]

# Display the Churn probabilities
print("\nChurn Probabilities:")
print(churn_probabilities)

test_X = test_df.drop(columns=['Churn'])
test_y = test_df['Churn']

predictions = []
for _, row in test_X.iterrows():
    evidence = row.to_dict()
    result = infer.map_query(variables=['Churn'], evidence=evidence)
    predictions.append(result['Churn'])

accuracy = accuracy_score(test_y, predictions)
print(f"\nModel Accuracy on Test Data: {accuracy:.4f}")